In [197]:
# write svm for classification using joint_angles_per_image.csv file
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the CSV file
df = pd.read_csv("joint_angles_and_distances_per_image.csv")

# Display the first few rows of the dataframe
df.head()




,Image,Nose-Left Eye_distance,Nose-Right Eye_distance,Nose-Left Ear_distance,Nose-Right Ear_distance,Nose-Left Shoulder_distance,Nose-Right Shoulder_distance,Nose-Left Elbow_distance,Nose-Right Elbow_distance,Nose-Left Wrist_distance,...,Right Hip_position_x,Left Knee_position_y,Left Knee_position_x,Right Knee_position_y,Right Knee_position_x,Left Ankle_position_y,Left Ankle_position_x,Right Ankle_position_y,Right Ankle_position_x,Label
0,368,12.476423,32.941528,1060.568970,104.924133,194.045792,210.504044,343.160065,290.698456,294.530273,...,715.714417,1381.440796,1080.000000,1607.054443,739.180786,1619.709229,1077.896484,1712.911865,769.4646,bench press
1,426,45.579567,16.659624,138.062668,789.113647,238.754913,204.484955,789.113647,789.113647,789.113647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,bench press
2,310,0.000000,0.000000,0.000000,0.000000,392.825195,329.930786,394.707245,270.524017,359.846985,...,281.136047,240.432083,469.397736,223.355743,349.675140,0.000000,0.000000,0.000000,0.0000,bench press
3,233,45.474056,23.322979,128.468124,875.496338,185.837234,74.269760,344.028046,122.538155,252.904373,...,95.073547,742.361389,55.407043,718.409668,77.764114,0.000000,0.000000,0.000000,0.0000,bench press
4,403,0.000000,0.000000,0.000000,0.000000,1218.988403,1079.262329,1393.242188,1015.033691,1173.195312,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,bench press


In [198]:
df.shape

(11455, 172)

In [199]:

df.isnull().sum().sort_values(ascending=False)

Image                                0
Right Wrist-Left Knee_distance       0
Left Wrist-Left Hip_distance         0
Left Wrist-Right Hip_distance        0
Left Wrist-Left Knee_distance        0
                                    ..
Left Ear-Right Ankle_distance        0
Right Ear-Left Shoulder_distance     0
Right Ear-Right Shoulder_distance    0
Right Ear-Left Elbow_distance        0
Label                                0
Length: 172, dtype: int64

In [200]:
# print dropped columns 

dropped_columns = df.columns[df.isnull().sum() > 0]
print(f"Dropped columns: {dropped_columns}")

Dropped columns: Index([], dtype='object')


In [201]:
# drop nan columns
df = df.dropna(axis=1)


In [202]:
df.shape

(11455, 172)

In [203]:
df.isnull().sum().sum()

0

In [204]:
# print df's class distribution
print(df['Label'].value_counts())

Label
bench press     2291
hammer curl     2291
lat pulldown    2291
plank           2291
push up         2291
Name: count, dtype: int64


In [205]:
# make all label's count equal to the minimum count
# randomly select rows from each class to make the count equal to the minimum count
min_count = df['Label'].value_counts().min()
dfs = []
for label in df['Label'].unique():
    dfs.append(df[df['Label'] == label].sample(min_count))
df = pd.concat(dfs)

# print df's class distribution
print(df['Label'].value_counts())
# save the new dataframe to a new csv file
df.to_csv("joint_angles_and_distances_per_image_balanced.csv", index=False)

Label
bench press     2291
hammer curl     2291
lat pulldown    2291
plank           2291
push up         2291
Name: count, dtype: int64


In [206]:
# encode the labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# save the encoder to use it later
import joblib

joblib.dump(le, "models/label_encoder.joblib")


['models/label_encoder.joblib']

In [207]:
# Split the data into features (X) and labels (y)
X = df.drop(columns=["Image", "Label"])  # Features
y = df["Label"]  # Labels

# normalize the data except the label using standard scaler
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X = standardScaler.fit_transform(X)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [208]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# train the model with xg boost
from xgboost import XGBClassifier


model = XGBClassifier()
model.fit(X_train, y_train)

# save the model
joblib.dump(model, "models/xgboost_model.joblib")

# Make predictions
y_pred = model.predict(X_test)

# classification report
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/xgboost_model.joblib")


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       469
           1       1.00      1.00      1.00       477
           2       0.99      0.99      0.99       461
           3       0.97      0.99      0.98       450
           4       1.00      0.96      0.98       434

    accuracy                           0.99      2291
   macro avg       0.99      0.99      0.99      2291
weighted avg       0.99      0.99      0.99      2291

[[467   0   1   0   1]
 [  0 476   1   0   0]
 [  1   0 457   2   1]
 [  2   0   1 447   0]
 [  3   2   0  11 418]]


['models/xgboost_model.joblib']

In [209]:
# train and test svm model
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/svm_model.joblib")



              precision    recall  f1-score   support

           0       0.93      0.97      0.95       469
           1       0.99      0.97      0.98       477
           2       0.98      0.95      0.96       461
           3       0.93      0.98      0.95       450
           4       1.00      0.93      0.96       434

    accuracy                           0.96      2291
   macro avg       0.96      0.96      0.96      2291
weighted avg       0.96      0.96      0.96      2291

[[456   2   5   5   1]
 [ 10 462   5   0   0]
 [  5   1 440  14   1]
 [  8   0   1 441   0]
 [ 13   2   0  14 405]]


['models/svm_model.joblib']

In [210]:
# train and test random forest model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/random_forest_model.joblib")


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       469
           1       1.00      1.00      1.00       477
           2       1.00      0.99      0.99       461
           3       0.97      0.99      0.98       450
           4       0.99      0.97      0.98       434

    accuracy                           0.99      2291
   macro avg       0.99      0.99      0.99      2291
weighted avg       0.99      0.99      0.99      2291

[[467   0   1   0   1]
 [  0 476   1   0   0]
 [  2   0 455   3   1]
 [  1   0   0 447   2]
 [  2   1   0  11 420]]


['models/random_forest_model.joblib']

In [211]:
# train mlp model
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(250, 150,75,50,25), max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# classifitaion report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# save the model
joblib.dump(model, "models/mlp_model.joblib")

Accuracy: 98.78%
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       469
           1       1.00      1.00      1.00       477
           2       0.98      0.99      0.99       461
           3       1.00      0.97      0.98       450
           4       0.97      0.99      0.98       434

    accuracy                           0.99      2291
   macro avg       0.99      0.99      0.99      2291
weighted avg       0.99      0.99      0.99      2291



['models/mlp_model.joblib']

# validation part

### load data

In [212]:
import pandas as pd
# Load the CSV file
df = pd.read_csv("validation_dataset.csv")


In [213]:
df.dropna(axis=1,inplace=True)

In [214]:
df.shape

(5831, 172)

In [215]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# encode the labels using LabelEncoder
# load the encoder
label_encoder = joblib.load("models/label_encoder.joblib")
df["Label"] = label_encoder.transform(df["Label"])

# Split the data into features (X) and labels (y)
X_val = df.drop(columns=["Image", "Label"])  # Features
y_val = df["Label"]  # Labels

# normalize the data except the label using standard scaler
# load model
standardScaler = joblib.load("models/standard_scaler.joblib")
X_val = standardScaler.fit_transform(X_val)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")




['models/standard_scaler.joblib']

In [216]:
X_val.shape

(5831, 170)

### test models

In [217]:
X_val.shape

(5831, 170)

In [218]:
# test the xgboost model
model = joblib.load("models/xgboost_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.53      0.81      0.64       742
           1       0.68      0.74      0.71       934
           2       0.53      0.46      0.49       832
           3       0.14      0.19      0.16       735
           4       0.79      0.60      0.68      2588

    accuracy                           0.58      5831
   macro avg       0.54      0.56      0.54      5831
weighted avg       0.62      0.58      0.59      5831

[[ 601   12   22   52   55]
 [ 101  695   18   90   30]
 [ 177  141  383   61   70]
 [  28  124  198  140  245]
 [ 223   43  105  674 1543]]


In [219]:
# test the svm model
model = joblib.load("models/svm_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.40      0.66      0.50       742
           1       0.68      0.81      0.74       934
           2       0.68      0.55      0.61       832
           3       0.00      0.00      0.00       735
           4       0.46      0.32      0.38      2588

    accuracy                           0.43      5831
   macro avg       0.44      0.47      0.44      5831
weighted avg       0.46      0.43      0.44      5831

[[489   0  27   5 221]
 [ 33 752  36  45  68]
 [ 68  71 457   9 227]
 [ 48 116 113   0 458]
 [591 162  40 968 827]]


In [220]:
# test the random forest model
model = joblib.load("models/random_forest_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.36      0.68      0.47       742
           1       0.69      0.49      0.57       934
           2       0.52      0.54      0.53       832
           3       0.00      0.00      0.00       735
           4       0.79      0.61      0.69      2588

    accuracy                           0.51      5831
   macro avg       0.47      0.46      0.45      5831
weighted avg       0.58      0.51      0.53      5831

[[ 504    0   24  159   55]
 [ 187  460  108   94   85]
 [  78   80  448   59  167]
 [ 490    2  120    0  123]
 [ 155  124  162  569 1578]]


In [221]:
# test the mlp model
model = joblib.load("models/mlp_model.joblib")
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))



              precision    recall  f1-score   support

           0       0.69      0.85      0.76       742
           1       0.63      0.84      0.72       934
           2       0.62      0.63      0.62       832
           3       0.13      0.19      0.16       735
           4       0.74      0.50      0.60      2588

    accuracy                           0.58      5831
   macro avg       0.56      0.60      0.57      5831
weighted avg       0.62      0.58      0.59      5831

[[ 633    9   49   23   28]
 [  38  783   41   54   18]
 [  29   81  525   31  166]
 [ 143  127   78  143  244]
 [  76  250  155  817 1290]]


In [222]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.optimizers import Adam

def train_model(X_train, y_train, X_test, y_test, input_shape, epochs=30, batch_size=32, validation_split=0.2, learning_rate=0.001):
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_shape,)),  # Input layer
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax') 
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=validation_split,  # Use a portion of training data for validation
        epochs=epochs,            # Number of epochs
        batch_size=batch_size,    # Batch size
        verbose=0               # Show training progress
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")



    return model, history, test_acc, test_loss


In [ ]:
# test the mlp model with validation data
# use grid search to find the best hyperparameters
best_model = None
best_accuracy = 0
best_params = None  

from sklearn.model_selection import ParameterGrid

param_grid = {
    "epochs": [30, 50],
    "batch_size": [32, 64, 128],
    "validation_split": [0.1, 0.2, 0.3],
    "learning_rate": [0.001, 0.005, 0.01]
}

for params in ParameterGrid(param_grid):
    model, history, test_acc, test_loss = train_model(X_train, y_train, X_val, y_val, X_train.shape[1])
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = model
        best_params = params

        





c:\Users\Samet\anaconda3\envs\aiclub\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Accuracy: 0.5347
Test Accuracy: 0.5129
Test Accuracy: 0.5210
Test Accuracy: 0.5474
Test Accuracy: 0.5092
Test Accuracy: 0.5265
Test Accuracy: 0.5105
Test Accuracy: 0.5376
Test Accuracy: 0.4821
Test Accuracy: 0.5467


KeyboardInterrupt: 

In [131]:
# try different model architectures

def train_model(X_train, y_train, X_test, y_test, input_shape, epochs=30, batch_size=32, validation_split=0.2, learning_rate=0.001):
    model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', input_shape=(input_shape,)),  # Input layer
        layers.Dropout(0.2),
        layers.Dense(256, activation='relu'),  # Input layer
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax') 
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=validation_split,  # Use a portion of training data for validation
        epochs=epochs,            # Number of epochs
        batch_size=batch_size,    # Batch size
        verbose=0               # Show training progress
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc:.4f}")



    return model, history, test_acc, test_loss

In [132]:
# test the mlp model with validation data
# use grid search to find the best hyperparameters
best_model = None
best_accuracy = 0
best_params = None

from sklearn.model_selection import ParameterGrid

param_grid = {
    "epochs": [30, 50],
    "batch_size": [32, 64, 128],
    "validation_split": [0.1, 0.2, 0.3],
    "learning_rate": [0.001, 0.005, 0.01]
}

for params in ParameterGrid(param_grid):
    model, history, test_acc, test_loss = train_model(X_train, y_train, X_val, y_val, X_train.shape[1])
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_model = model
        best_params = params

        





c:\Users\Samet\anaconda3\envs\aiclub\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Accuracy: 0.5570


KeyboardInterrupt: 